In [ ]:
## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using StatsBase
using Distributions
using Plots

In [2]:
using TensorFlow
using MLDataUtils

In [3]:
using DensityEstimationML

In [4]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    conditioner::Tensor
    t::Tensor
    pdf::Tensor
end

In [5]:
function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    ts = reshape([t], (1,1))
    pdf(est, ts) |> first
end

function Distributions.pdf(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, est.pdf, Dict(est.t=>ts')) |> vec
end

function Distributions.loglikelihood(est::NeuralDensityEstimator, ts::AbstractVector)
    gr = est.sess.graph
    run(est.sess, gr["loglikelihood"], Dict(est.t=>ts')) |> vec
end


In [6]:
X = [20.0, 25.0, 27.0, 28.0]
Y = [30.0, 30.0, 24, 27.]
plot(X,Y)
push!(X, 25)


5-element Array{Float64,1}:
 20.0
 25.0
 27.0
 28.0
 25.0

In [7]:
ignore(epoch, arg...) = nothing

function StatsBase.fit!(estimator::NeuralDensityEstimator, observations, callback=ignore;
    epochs = 20)
    
    gr = estimator.sess.graph
    for ii in 1:epochs
        outs = run(estimator.sess, 
            [gr["ysmin"],gr["ysmax"],gr["loglikelihood"], gr["working_loss"],
                estimator.optimizer],
            Dict(estimator.t=>observations'))
        ii % 100 == 1 && callback(ii, outs...)
    end
    estimator
end

In [8]:
function NeuralDensityEstimator(prob_layer_sizes, support)
    sess = Session(Graph())
    @tf begin
        t = placeholder(Float32, shape=[1, -1])
        smin = constant(reshape([minimum(support)],(1,1)))
        smax = constant(reshape([maximum(support)],(1,1)))
        
        layer_sizes= [1; prob_layer_sizes; 1]
        
        network_fun_stack = Function[Base.identity]       
       
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
                       
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            Wii2  = Ops.mul(Wii, Wii; name = "W_$(ii)_squared")
            
            #bii = get_variable("b_$ii", [above_size, 1], Float32)
            #act_fun = z -> nn.sigmoid(Wii2*z .+ bii)
            
            act_fun = if ii!=length(layer_sizes)
                bii = get_variable("b_$ii", [above_size, 1], Float32)
                z -> nn.sigmoid(Wii2*z .+ bii)
            else
                z-> exp(Wii2*z)
            end
            push!(network_fun_stack, z->act_fun(network_fun_stack[ii-1](z)))
        end
        
        network = network_fun_stack[end]

        
        ysmin = TensorFlow.identity(network(smin))
        ysmax = TensorFlow.identity(network(smax))
        yt = network(t)
        
        denominator = (ysmax-ysmin) #area
        numerator = gradients(yt,t)
        pdf = numerator/denominator
        
        
        n_points = TensorFlow.shape(t)[2]
        loglikelihood = reduce_sum(log(numerator)) - n_points.*log(denominator)
        
        area_loss = (1f0.-denominator)^2
        working_loss = -1*loglikelihood + 0.1*area_loss
        
        optimizer = train.minimize(train.AdamOptimizer(), working_loss)
        
        
        # Conditioning
        # Make sure that ysmin~=1, and ysmax~=2
        condition_loss = (1f0 - ysmin)^2 + (2f0 - ysmax)^2
        condition_optimiser = train.minimize(train.AdamOptimizer(;name="adam_cond"), condition_loss)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, condition_optimiser, t, pdf)
end

NeuralDensityEstimator

In [9]:
"""
    condition(est::NeuralDensityEstimator tol = 1e-15, max_epochs=2_000)
    
"Conditions" the neural density estimate so the support extrema are mapped to 1. and 2.
This improves training by adjusting the area the network has the learn over

"""
function condition!(est::NeuralDensityEstimator, tol = 1e-15, max_epochs=2_000)
    gr = est.sess.graph
    for ii in 1:2_000
        _, ysmin, ysmax, condition_loss = run(est.sess, [est.conditioner, gr["ysmin"],gr["ysmax"], gr["condition_loss"]])
        ii % 50 == 1 && @show (ii, ysmin, ysmax, condition_loss)
        if condition_loss[1] < 1e-15
            break
        end
    end
end


condition!

In [10]:
function demonstration_plot(est, dataset, data=rand(dataset), args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset))
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    estimated_loglikelihood = loglikelihood(dataset, data)
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

demonstration_plot (generic function with 2 methods)

In [11]:
function demo(dataset, layers, epochs=20_000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))
    condition!(est)
    println("Conditioning Done")
    fit!(est, data; epochs=epochs)
    println("Fitting Done")
    demonstration_plot(est, dataset, data)
end

demo (generic function with 2 methods)

In [12]:
demo(GenerateDatasets.Likas1(), [64,64], 20_000)

loglikelihood(dataset, data) = -10516.625172431352


2017-09-15 13:19:30.323654: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-15 13:19:30.323686: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-15 13:19:30.323692: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-15 13:19:30.507198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-15 13:19:30.507737: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

(ii, ysmin, ysmax, condition_loss) = (1, [1.00271], [1.00271], [0.994586])
(ii, ysmin, ysmax, condition_loss) = (51, [1.16601], [1.16641], [0.722425])
(ii, ysmin, ysmax, condition_loss) = (101, [1.51874], [1.54791], [0.47348])
(ii, ysmin, ysmax, condition_loss) = (151, [1.49838], [1.61576], [0.396028])
(ii, ysmin, ysmax, condition_loss) = (201, [1.46333], [1.71975], [0.293216])
(ii, ysmin, ysmax, condition_loss) = (251, [1.42282], [1.79745], [0.219805])
(ii, ysmin, ysmax, condition_loss) = (301, [1.38944], [1.84106], [0.176923])
(ii, ysmin, ysmax, condition_loss) = (351, [1.36478], [1.86682], [0.150798])
(ii, ysmin, ysmax, condition_loss) = (401, [1.34577], [1.88398], [0.133016])
(ii, ysmin, ysmax, condition_loss) = (451, [1.33023], [1.89659], [0.119748])
(ii, ysmin, ysmax, condition_loss) = (501, [1.31692], [1.9065], [0.10918])
(ii, ysmin, ysmax, condition_loss) = (551, [1.3051], [1.91467], [0.100369])
(ii, ysmin, ysmax, condition_loss) = (601, [1.29434], [1.92165], [0.0927774])
(ii, 

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 0 
 
 
 10 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.8117,175.813 39.0374,175.813 39.263,175.813 39.4886,175.813 39.7142,175.813 39.9399,175.813 40.1655,175.813 40.3911,175.813 40.6167,175.813 40.8424,175.813 
 41.068,175.813 41.2936,175.813 41.5193,175.813 41.7449,175.813 41.9705,175.813 42.1961,175.813 42.4218,175.813 42.6474,175.813 42.873,175.813 43.0986,175.813 
 43.3243,175.813 43.5499,175.813 43.7755,175.813 44.0011,175.813 44.2268,175.813 44.4524,175.813 44.678,175.813 44.9036,175.813 45.1293,175.813 45.3549,175.813 
 45.5805,175.813 45.8062,175.813 46.0318,175.813 46.2574,175.813 46.483,175.813 46.7087,175.813 46.9343,175.813 47.1599,175.813 47.3855,175.813 47.6112,175.813 
 47.8368,175.813 48.0624,175.813 48.288,175.813 48.5137,175.813 48.7393,175.813 48.9649,175.813 49.1905,175.813 49.4162,175.813 49.6418,175.813 49.8674,175.813 
 50.0931,175.813 50.3187,175.813 50.5443,175.813 50.7699,175.813 50.9956,175.813 51.2212,175.813 51.4468,175.813 51.6724,175.813 51.8981,175.813 52.1237,175.813 
 52.3493,175.813 52.5749,175.813 52.8006,175.813 53.0262,175.813 53.2518,175.813 53.4774,175.813 53.7031,175.813 53.9287,175.813 54.1543,175.813 54.38,175.813 
 54.6056,175.813 54.8312,175.813 55.0568,175.813 55.2825,175.813 55.5081,175.813 55.7337,175.813 55.9593,175.813 56.185,175.813 56.4106,175.813 56.6362,175.813 
 56.8618,175.813 57.0875,175.813 57.3131,175.813 57.5387,175.813 57.7643,175.813 57.99,175.813 58.2156,175.813 58.4412,175.813 58.6669,175.813 58.8925,175.813 
 59.1181,175.813 59.3437,175.813 59.5694,175.813 59.795,175.813 60.0206,175.813 60.2462,175.813 60.4719,175.813 60.6975,175.813 60.9231,175.813 61.1487,175.813 
 61.3744,175.813 61.6,175.813 61.8256,175.813 62.0512,175.813 62.2769,175.813 62.5025,175.813 62.7281,175.813 62.9538,175.813 63.1794,175.813 63.405,175.813 
 63.6306,175.813 63.8563,175.813 64.0819,175.813 64.3075,175.813 64.5331,175.813 64.7588,175.813 64.9844,175.813 65.21,175.813 65.4356,175.813 65.6613,175.813 
 65.8869,175.813 66.1125,175.813 66.3381,175.813 66.5638,175.813 66.7894,175.813 67.015,175.813 67.2407,175.813 67.4663,175.813 67.6919,175.813 67.9175,175.813 
 68.1432,175.813 68.3688,175.812 68.5944,175.812 68.82,175.812 69.0457,175.812 69.2713,175.812 69.4969,175.812 69.7225,175.812 69.9482,175.812 70.1738,175.812 
 70.3994,175.812 70.625,175.812 70.8507,175.812 71.0763,175.812 71.3019,175.812 71.5276,175.812 71.7532,175.812 71.9788,175.812 72.2044,175.812 72.4301,175.812 
 72.6557,175.812 72.8813,175.812 73.1069,175.812 73.3326,175.812 73.5582,175.812 73.7838,175.812 74.0094,175.812 74.2351,175.812 74.4607,175.812 74.6863,175.812 
 74.912,175.812 75.1376,175.811 75.3632,175.811 75.5888,175.811 75.8145,175.811 76.0401,175.811 76.2657,175.811 76.4913,175.811 76.717,175.811 76.9426,175.811 
 77.1682,175.811 77.3938,175.811 77.6195,175.811 77.8451,175.811 78.0707,175.811 78.2963,175.811 78.522,175.81 78.7476,175.81 78.9732,175.81 79.1989,175.81 
 79.4245,175.81 79.6501,175.81 79.8757,175.81 80.1014,175.81 80.327,175.81 80.5526,175.81 80.7782,175.809 81.0039,175.809 81.2295,175.809 81.4551,175.809 
 81.6807,175.809 81.9064,175.809 82.132,175.809 82.3576,175.809 82.5832,175.808 82.8089,175.808 83.0345,175.808 83.2601,175.808 83.4858,175.808 83.7114,175.808 
 83.937,175.807 84.1626,175.807 84.3883,175.807 84.6139,175.807 84.8395,175.807 85.0651,175.806 85.2908,175.806 85.5164,175.806 85.742,175.806 85.9676,175.805 
 86.1933,175.805 86.4189,175.805 86.6445,175.805 86.8701,175.804 87.0958,175.804 87.3214,175.804 87.547,175.803 87.7727,175.803 87.9983,175.803 88.2239,175.802 
 88.4495,175.802 88.6752,175.802 88.9008,175.801 89.1264,175.801 89.352,175.8 89.5777,175.8 89.8033,175.799 90.0289,175.799 90.2545,175.798 90.4802,175.798 
 90.7058,175.797 90.9314,175.

In [ ]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

In [ ]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 20_000)

In [ ]:
demo(Arcsine(1,4), [64,64], 20_000)

In [ ]:
dataset = GenerateDatasets.Likas1()
data = original_sample(dataset)
@show loglikelihood(dataset, data)
est = NeuralDensityEstimator([64], approximate_support(dataset))
condition!(est)
println("Conditioning Done")
fit!(est, data; epochs=10_000)
println("Fitting Done")

In [ ]:
gr =est.sess.graph
run(est.sess, exp(6*gr["W_2"].^2 + gr["b_2"]))

In [ ]:
collect(keys(est.sess.graph))